# Pre-Processing Agent

- This notebook helps you generate needed prompts for the LLM. 
- You still need access to an LLM and execute the code provided by the LLM.

In [144]:
import os
import io
import pandas as pd

def gather_summary(df):
    # 2. Capture df.info() output
    buffer = io.StringIO()
    df.info(buf=buffer)
    info_text = buffer.getvalue()
    print(info_text)

    # 3. Calculate missing value stats
    missing_stats = (df.isna().sum() / len(df) * 100).sort_values(ascending=False)
    missing_summary = "\n".join([f"{col}: {val:.2f}%" for col, val in missing_stats.items()])
    print(missing_summary)

    # 4. Get column data types
    column_types = "\n".join([f"{col}: {dtype}" for col, dtype in df.dtypes.items()])
    print(column_types)

    # 5. Get unique value counts
    unique_counts = df.nunique()  # Will no longer fail on unhashable dict
    unique_counts_summary = "\n".join([f"{col}: {count}" for col, count in unique_counts.items()])
    print(unique_counts_summary)

    summary_text = f"""
    Dataset Name: {dataset_name}
    ----------------------------
    Shape: {df.shape[0]} rows x {df.shape[1]} columns
    
    Column Data Types:
    {column_types}
    
    Missing Value Percentage:
    {missing_summary}
    
    Unique Value Counts:
    {unique_counts_summary}
    
    Data (first {n_sample} rows):
    {df.head(n_sample).to_string()}
    
    Data Description:
    {df.describe().to_string()}
    
    Data Info:
    {info_text}
    """
    print(summary_text)

    return summary_text

In [145]:
dataset_name = "vgsales.csv"

In [146]:
n_sample=30
summary_text = ""
if os.path.exists(dataset_name):
    df = pd.read_csv(dataset_name)
    df = df.convert_dtypes()
    summary_text = gather_summary(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  Int64  
 1   Name          16598 non-null  string 
 2   Platform      16598 non-null  string 
 3   Year          16327 non-null  Int64  
 4   Genre         16598 non-null  string 
 5   Publisher     16540 non-null  string 
 6   NA_Sales      16598 non-null  Float64
 7   EU_Sales      16598 non-null  Float64
 8   JP_Sales      16598 non-null  Float64
 9   Other_Sales   16598 non-null  Float64
 10  Global_Sales  16598 non-null  Float64
dtypes: Float64(5), Int64(2), string(4)
memory usage: 1.5 MB

Year: 1.63%
Publisher: 0.35%
Rank: 0.00%
Name: 0.00%
Platform: 0.00%
Genre: 0.00%
NA_Sales: 0.00%
EU_Sales: 0.00%
JP_Sales: 0.00%
Other_Sales: 0.00%
Global_Sales: 0.00%
Rank: Int64
Name: string
Platform: string
Year: Int64
Genre: string
Publisher: string
NA_Sales: Float

# Data Cleaning Expert Agent	

- Collects dataset characteristics from your dataset
- Prompts LLM for which steps to perform based on the data characteristics


In [147]:
if summary_text:    
    prompt=f"""
    You are a Data Cleaning Expert. Given the following information about the data, 
    recommend a series of numbered steps to take to clean and preprocess it. 
    The steps should be tailored to the data characteristics and should be helpful 
    for a data cleaning agent that will be implemented.
    
    General Steps:
    Things that should be considered in the data cleaning steps:
    
    * Removing columns if more than 40 percent of the data is missing
    * Imputing missing values with the mean of the column if the column is numeric
    * Imputing missing values with the mode of the column if the column is categorical
    * Converting columns to the correct data type
    * Removing duplicate rows
    * Removing rows with missing values
    * Removing rows with extreme outliers (3X the interquartile range)
    
    Custom Steps:
    * Analyze the data to determine if any additional data cleaning steps are needed.
    * Recommend steps that are specific to the data provided. Include why these steps are necessary or beneficial.
    * If no additional steps are needed, simply state that no additional steps are required.
    
    IMPORTANT:
    Make sure to take into account any additional user instructions that may add, remove or modify some of these steps. Include comments in your code to explain your reasoning for each step. Include comments if something is not done because a user requested. Include comments if something is done because a user requested.
    
    Below are summaries of all datasets provided:
    {summary_text}
    
    Return steps as a numbered list. You can return short code snippets to demonstrate actions. But do not return a fully coded solution. The code will be generated separately by a Coding Agent.
    Fill this list : recommended_steps = [
    ]

    Avoid these:
    1. Do not include steps to save files.
    2. Do not include unrelated user instructions that are not related to the data cleaning.
    """

    print(prompt)
    


    You are a Data Cleaning Expert. Given the following information about the data, 
    recommend a series of numbered steps to take to clean and preprocess it. 
    The steps should be tailored to the data characteristics and should be helpful 
    for a data cleaning agent that will be implemented.
    
    General Steps:
    Things that should be considered in the data cleaning steps:
    
    * Removing columns if more than 40 percent of the data is missing
    * Imputing missing values with the mean of the column if the column is numeric
    * Imputing missing values with the mode of the column if the column is categorical
    * Converting columns to the correct data type
    * Removing duplicate rows
    * Removing rows with missing values
    * Removing rows with extreme outliers (3X the interquartile range)
    
    Custom Steps:
    * Analyze the data to determine if any additional data cleaning steps are needed.
    * Recommend steps that are specific to the data provided. 

## Now, you must prompt an LLM 

- Ask for which steps to perform based on the data characteristics
- Then, manually extract suggested steps and add them to the following list.

In [148]:
recommended_steps = [
    "Load the CSV into a pandas DataFrame and immediately create a copy to avoid mutating the raw data.",
    "Strip whitespace from column names to ensure consistent and bug-free column access.",
    "Remove fully duplicated rows across all columns (even though Rank is unique, this is a safe sanity step).",
    "Check percentage of missing values per column and confirm that no column exceeds the 40% threshold, so no columns are dropped.",
    "Impute missing values in the numeric column 'Year' using the mean of the column, since missingness is low (~1.6%).",
    "Impute missing values in the categorical column 'Publisher' using the mode of the column, as missingness is minimal (~0.35%).",
    "Convert columns to appropriate data types, ensuring 'Year' is integer and sales columns are numeric.",
    "Validate internal consistency by comparing 'Global_Sales' to the sum of regional sales (NA_Sales + EU_Sales + JP_Sales + Other_Sales).",
    "If discrepancies are found between Global_Sales and the summed regional sales, replace Global_Sales with the computed sum to ensure consistency.",
    "Detect and remove extreme outliers in sales-related numeric columns using the 3×IQR rule to reduce skew from extreme values.",
    "Do not remove rows with missing values globally, because targeted imputation already handled the limited missing data.",
    "Optionally convert low-cardinality string columns such as 'Platform' and 'Genre' to categorical dtype for memory and performance benefits."
]


In [149]:
function_name = "clean_data"

if summary_text and len(recommended_steps) > 0:
    steps_text = "\n".join(map(str, recommended_steps))
    
    prompt2=f"""You are a Data Cleaning Agent. Your job is to create a {function_name}() function that can be run on the data 
    provided using the following recommended steps.
    
    Recommended Steps:
    {steps_text}
    
    You can use Pandas, Numpy, and Scikit Learn libraries to clean the data.
    
    Below are summaries of all datasets provided. Use this information about the data to help determine how to 
    clean the data:
    
    {summary_text}
    
    Return Python code in ```python``` format with a single function definition, {function_name}(data_raw), that 
    includes all imports inside the function.
    
    Return code to provide the data cleaning function:
    
    def {function_name}(data_raw):
        import pandas as pd
        import numpy as np
        ...
        return data_cleaned
    
    Best Practices and Error Preventions:
    
    Always ensure that when assigning the output of fit_transform() from SimpleImputer to a Pandas DataFrame 
    column, you call .ravel() or flatten the array, because fit_transform() returns a 2D array while a DataFrame column is 1D
    """
        
    print(prompt2)

You are a Data Cleaning Agent. Your job is to create a clean_data() function that can be run on the data 
    provided using the following recommended steps.
    
    Recommended Steps:
    Load the CSV into a pandas DataFrame and immediately create a copy to avoid mutating the raw data.
Strip whitespace from column names to ensure consistent and bug-free column access.
Remove fully duplicated rows across all columns (even though Rank is unique, this is a safe sanity step).
Check percentage of missing values per column and confirm that no column exceeds the 40% threshold, so no columns are dropped.
Impute missing values in the numeric column 'Year' using the mean of the column, since missingness is low (~1.6%).
Impute missing values in the categorical column 'Publisher' using the mode of the column, as missingness is minimal (~0.35%).
Convert columns to appropriate data types, ensuring 'Year' is integer and sales columns are numeric.
Validate internal consistency by comparing 'Global_S

## Python Clean Data Code from LLM


In [150]:
def clean_data(data_raw):
    import pandas as pd
    import numpy as np

    # ----------------------------
    # 0) Load into DataFrame + copy
    # ----------------------------
    if isinstance(data_raw, pd.DataFrame):
        df = data_raw.copy(deep=True)
    else:
        try:
            df = pd.read_csv(data_raw).copy(deep=True)
        except Exception:
            from io import StringIO
            df = pd.read_csv(StringIO(str(data_raw))).copy(deep=True)

    # ----------------------------
    # 1) Strip whitespace from column names
    # ----------------------------
    df.columns = df.columns.astype(str).str.strip()

    # ----------------------------
    # 2) Remove fully duplicated rows
    # ----------------------------
    df = df.drop_duplicates().reset_index(drop=True)

    # ----------------------------
    # 3) Missingness check (40% rule safety)
    # ----------------------------
    missing_pct = df.isna().mean()
    drop_cols = missing_pct[missing_pct > 0.40].index
    if len(drop_cols) > 0:
        df = df.drop(columns=list(drop_cols))

    # ----------------------------
    # 4) Clean string columns
    # ----------------------------
    for col in ["Name", "Platform", "Genre", "Publisher"]:
        if col in df.columns:
            df[col] = df[col].astype("string").str.strip()
            df.loc[df[col] == "", col] = pd.NA

    # ----------------------------
    # 5) Impute Year with mean (handle pandas Int64 properly)
    # ----------------------------
    if "Year" in df.columns:
        # Convert to numeric then FORCE float so we can fill with a float mean
        year = pd.to_numeric(df["Year"], errors="coerce").astype("float64")

        year_mean = float(year.mean())  # scalar float
        year = year.fillna(year_mean)

        # Enforce plausible bounds (based on dataset summary)
        year = year.where((year >= 1980) & (year <= 2020), year_mean)

        # Round and convert to integer
        df["Year"] = np.rint(year).astype("int64")

    # ----------------------------
    # 6) Impute Publisher with mode
    # ----------------------------
    if "Publisher" in df.columns:
        publisher_mode = df["Publisher"].mode(dropna=True)
        if not publisher_mode.empty:
            df["Publisher"] = df["Publisher"].fillna(publisher_mode.iloc[0])

    # ----------------------------
    # 7) Convert sales columns to numeric
    # ----------------------------
    sales_cols = ["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales", "Global_Sales"]
    for col in sales_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # No negative sales; fill missing regional sales with 0
    regional_cols = ["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"]
    for col in regional_cols:
        if col in df.columns:
            df.loc[df[col] < 0, col] = np.nan
            df[col] = df[col].fillna(0.0)

    # If Global_Sales has any NaN (shouldn't, but safe), fill with 0 for now
    if "Global_Sales" in df.columns:
        df.loc[df["Global_Sales"] < 0, "Global_Sales"] = np.nan
        df["Global_Sales"] = df["Global_Sales"].fillna(0.0)

    # ----------------------------
    # 8) Validate & fix Global_Sales consistency
    # ----------------------------
    needed = set(sales_cols)
    if needed.issubset(df.columns):
        regional_sum = df["NA_Sales"] + df["EU_Sales"] + df["JP_Sales"] + df["Other_Sales"]

        # allow small rounding tolerance
        tolerance = 0.01
        mismatch = (df["Global_Sales"] - regional_sum).abs() > tolerance
        df.loc[mismatch, "Global_Sales"] = regional_sum.loc[mismatch]

    # ----------------------------
    # 9) Remove extreme outliers (3×IQR) on sales columns
    # ----------------------------
    mask_keep = pd.Series(True, index=df.index)
    for col in sales_cols:
        if col not in df.columns:
            continue

        s = df[col].astype("float64")
        if s.notna().sum() < 10:
            continue

        q1 = s.quantile(0.25)
        q3 = s.quantile(0.75)
        iqr = q3 - q1
        if pd.isna(iqr) or iqr == 0:
            continue

        lower = q1 - 3 * iqr
        upper = q3 + 3 * iqr
        mask_keep &= s.between(lower, upper, inclusive="both")

    df = df.loc[mask_keep].reset_index(drop=True)

    # ----------------------------
    # 10) Optional categorical optimization
    # ----------------------------
    for col in ["Platform", "Genre"]:
        if col in df.columns:
            df[col] = df[col].astype("category")

    # ----------------------------
    # 11) Final dtype cleanup
    # ----------------------------
    if "Rank" in df.columns:
        df["Rank"] = pd.to_numeric(df["Rank"], errors="coerce").round().astype("int64")

    if "Name" in df.columns:
        df["Name"] = df["Name"].astype("string")
    if "Publisher" in df.columns:
        df["Publisher"] = df["Publisher"].astype("string")

    return df


## Summary Creation


In [151]:
# ================================
# FINAL SUBMISSION CELL
# ================================
# Assumes:
# 1) df is the original DataFrame
# 2) clean_data(df) is already defined above

import pandas as pd

# ---------- Helper: Summary Statistics ----------
def summary_stats(df):
    # Numeric summary
    num = df.describe(include=["number"]).T

    # Categorical summary (only if present)
    cat_cols = df.select_dtypes(include=["object", "string", "category"]).columns
    if len(cat_cols) > 0:
        cat = df[cat_cols].describe().T
    else:
        cat = pd.DataFrame()

    # General overview
    overview = pd.DataFrame({
        "dtype": df.dtypes.astype(str),
        "missing_count": df.isna().sum(),
        "missing_pct": (df.isna().mean() * 100).round(2),
        "n_unique": df.nunique(dropna=False),
    })

    return num, cat, overview


# ---------- BEFORE CLEANING ----------
num_before, cat_before, overview_before = summary_stats(df)

print("===== BEFORE CLEANING: NUMERIC SUMMARY =====")
display(num_before)

if not cat_before.empty:
    print("===== BEFORE CLEANING: CATEGORICAL SUMMARY =====")
    display(cat_before)

print("===== BEFORE CLEANING: OVERVIEW =====")
display(overview_before.sort_values("missing_count", ascending=False))


# ---------- CLEAN DATA ----------
df_clean = clean_data(df)


# ---------- AFTER CLEANING ----------
num_after, cat_after, overview_after = summary_stats(df_clean)

print("===== AFTER CLEANING: NUMERIC SUMMARY =====")
display(num_after)

if not cat_after.empty:
    print("===== AFTER CLEANING: CATEGORICAL SUMMARY =====")
    display(cat_after)

print("===== AFTER CLEANING: OVERVIEW =====")
display(overview_after.sort_values("missing_count", ascending=False))


# ---------- SAVE FILES FOR SUBMISSION ----------
# Original & cleaned CSV
df.to_csv("data_original.csv", index=False)
df_clean.to_csv("data_cleaned.csv", index=False)

# Summary statistics CSVs (optional but recommended)
num_before.to_csv("summary_numeric_before.csv")
num_after.to_csv("summary_numeric_after.csv")
overview_before.to_csv("summary_overview_before.csv")
overview_after.to_csv("summary_overview_after.csv")

print("===================================")
print("FILES CREATED FOR SUBMISSION:")
print("- data_original.csv")
print("- data_cleaned.csv")
print("- summary_numeric_before.csv")
print("- summary_numeric_after.csv")
print("- summary_overview_before.csv")
print("- summary_overview_after.csv")
print("===================================")
print("Original shape:", df.shape)
print("Cleaned shape:", df_clean.shape)


===== BEFORE CLEANING: NUMERIC SUMMARY =====


,count,mean,std,min,25%,50%,75%,max
Rank,16598.0,8300.605254,4791.853933,1.0,4151.25,8300.5,12449.75,16600.0
Year,16327.0,2006.406443,5.828981,1980.0,2003.0,2007.0,2010.0,2020.0
NA_Sales,16598.0,0.264667,0.816683,0.0,0.0,0.08,0.24,41.49
EU_Sales,16598.0,0.146652,0.505351,0.0,0.0,0.02,0.11,29.02
JP_Sales,16598.0,0.077782,0.309291,0.0,0.0,0.0,0.04,10.22
Other_Sales,16598.0,0.048063,0.188588,0.0,0.0,0.01,0.04,10.57
Global_Sales,16598.0,0.537441,1.555028,0.01,0.06,0.17,0.47,82.74


===== BEFORE CLEANING: CATEGORICAL SUMMARY =====


,count,unique,top,freq
Name,16598,11493,Need for Speed: Most Wanted,12
Platform,16598,31,DS,2163
Genre,16598,12,Action,3316
Publisher,16540,578,Electronic Arts,1351


===== BEFORE CLEANING: OVERVIEW =====


...

===== AFTER CLEANING: CATEGORICAL SUMMARY =====


,count,unique,top,freq
Name,13494,9603,LEGO Jurassic World,8
Platform,13494,31,DS,1906
Genre,13494,12,Action,2768
Publisher,13494,561,Electronic Arts,1063


===== AFTER CLEANING: OVERVIEW =====


,dtype,missing_count,missing_pct,n_unique
Rank,int64,0,0.0,13494
Name,string,0,0.0,9603
Platform,category,0,0.0,31
Year,int64,0,0.0,38
Genre,category,0,0.0,12
Publisher,string,0,0.0,561
NA_Sales,Float64,0,0.0,97
EU_Sales,Float64,0,0.0,45
JP_Sales,Float64,0,0.0,17
Other_Sales,Float64,0,0.0,17


FILES CREATED FOR SUBMISSION:
- data_original.csv
- data_cleaned.csv
- summary_numeric_before.csv
- summary_numeric_after.csv
- summary_overview_before.csv
- summary_overview_after.csv
Original shape: (16598, 11)
Cleaned shape: (13494, 11)


## Fix Code Agent

In case of an error, provide stack-trace to LLM and re-run code

In [152]:
code_snippet = """"""
error = """"""
if code_snippet and error:
    data_cleaner_prompt = f"""
            You are a Fix Code Agent. Your job is to create a {function_name}() function that can be run on the data provided. 
            The function is currently broken and needs to be fixed.
            
            Make sure to only return the function definition for {function_name}().
            
            Return Python code in ```python``` format with a single function definition, {function_name}(data_raw), 
            that includes all imports inside the function.
            
            This is the broken code (please fix): 
            {code_snippet}
    
            Last Known Error:
            {error}
            """
    print(data_cleaner_prompt)

<hr> 

# Submit 6-fold via Moodle:
- The executed notebook 
- A html export of the executed notebook
- Original and cleaned CSV file
- **Summary Statistics before and after cleaning!**

### You must hand in this exercise via moodle